In [1]:
import string
import math
import time
from collections import Counter
import itertools
import gradio as gr
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class CustomCipher:
    def __init__(self):
        self.alphabet = string.ascii_uppercase
        self.n = len(self.alphabet)

    def _key_to_shifts(self, key):
        """Convert key string to shift values"""
        clean_key = ''.join([k.upper() for k in key if k.upper() in self.alphabet])
        if not clean_key:
            return [3]
        return [self.alphabet.index(k) for k in clean_key]

    # ==================== STAGE 1: VIGENÈRE CIPHER ====================
    def vigenere_encrypt(self, text, key):
        """First stage: Vigenère cipher encryption"""
        result = []
        key_shifts = self._key_to_shifts(key)

        for i, char in enumerate(text.upper()):
            if char in self.alphabet:
                shift = key_shifts[i % len(key_shifts)]
                new_char = self.alphabet[(self.alphabet.index(char) + shift) % self.n]
                result.append(new_char)
            else:
                result.append(char)
        return ''.join(result)

    def vigenere_decrypt(self, text, key):
        """Reverse of Vigenère cipher"""
        result = []
        key_shifts = self._key_to_shifts(key)

        for i, char in enumerate(text.upper()):
            if char in self.alphabet:
                shift = key_shifts[i % len(key_shifts)]
                new_char = self.alphabet[(self.alphabet.index(char) - shift) % self.n]
                result.append(new_char)
            else:
                result.append(char)
        return ''.join(result)

    # ==================== STAGE 2: SUBSTITUTION CIPHER ====================
    def substitution_encrypt(self, text, key):
        """Second stage: Simple substitution cipher"""
        # Create substitution mapping based on key
        key_chars = []
        for char in key.upper():
            if char in self.alphabet and char not in key_chars:
                key_chars.append(char)

        # Add remaining alphabet
        for char in self.alphabet:
            if char not in key_chars:
                key_chars.append(char)

        # Create substitution mapping
        substitution_map = {}
        for i, char in enumerate(self.alphabet):
            substitution_map[char] = key_chars[i]

        # Apply substitution
        result = []
        for char in text.upper():
            if char in self.alphabet:
                result.append(substitution_map[char])
            else:
                result.append(char)
        return ''.join(result)

    def substitution_decrypt(self, text, key):
        """Reverse of substitution cipher"""
        # Create substitution mapping based on key
        key_chars = []
        for char in key.upper():
            if char in self.alphabet and char not in key_chars:
                key_chars.append(char)

        # Add remaining alphabet
        for char in self.alphabet:
            if char not in key_chars:
                key_chars.append(char)

        # Create reverse mapping
        reverse_map = {}
        for i, char in enumerate(self.alphabet):
            reverse_map[key_chars[i]] = char

        # Apply reverse substitution
        result = []
        for char in text.upper():
            if char in self.alphabet:
                result.append(reverse_map[char])
            else:
                result.append(char)
        return ''.join(result)

    # ==================== STAGE 3: COLUMNAR TRANSPOSITION ====================
    def columnar_transposition(self, text, key):
        """Third stage: Columnar transposition - handles spaces by replacing them"""
        # Replace spaces with a special character for transposition
        temp_text = text.upper().replace(' ', '_')
        clean_text = ''.join([c for c in temp_text if c in self.alphabet or c == '_'])

        if not clean_text:
            return ""

        clean_key = ''.join([k.upper() for k in key if k.upper() in self.alphabet])
        if not clean_key:
            clean_key = "DEFAULTKEY"

        num_cols = len(clean_key)
        num_rows = math.ceil(len(clean_text) / num_cols)
        padded_text = clean_text.ljust(num_rows * num_cols, 'X')

        # Create grid
        grid = [list(padded_text[i:i+num_cols]) for i in range(0, len(padded_text), num_cols)]

        # Get column order
        key_order = sorted(range(num_cols), key=lambda k: clean_key[k])

        # Read columns in key order
        ciphertext = []
        for col in key_order:
            for row in range(num_rows):
                ciphertext.append(grid[row][col])

        return ''.join(ciphertext)

    def columnar_transposition_decrypt(self, text, key):
        """Reverse of columnar transposition - restores spaces"""
        clean_key = ''.join([k.upper() for k in key if k.upper() in self.alphabet])
        if not clean_key:
            clean_key = "DEFAULTKEY"

        num_cols = len(clean_key)
        num_rows = len(text) // num_cols

        # Get column order
        key_order = sorted(range(num_cols), key=lambda k: clean_key[k])

        # Reconstruct grid
        grid = [[''] * num_cols for _ in range(num_rows)]

        # Fill columns in key order
        idx = 0
        for col in key_order:
            for row in range(num_rows):
                if idx < len(text):
                    grid[row][col] = text[idx]
                    idx += 1

        # Read row-wise
        plaintext = ''.join(''.join(row) for row in grid)
        plaintext = plaintext.rstrip('X')

        # Restore spaces
        plaintext = plaintext.replace('_', ' ')
        return plaintext

    # ==================== FULL ENCRYPTION/DECRYPTION ====================
    def encrypt(self, plaintext, key, verbose=False):
        """Full encryption: Vigenère → Substitution → Columnar Transposition"""
        if len(key) < 10:
            raise ValueError("Key must be at least 10 characters long")

        stages = {}
        timing_data = {}

        # Stage 1: Vigenère (preserves spaces)
        start_time = time.time()
        stage1 = self.vigenere_encrypt(plaintext, key)
        timing_data['vigenere_encrypt'] = time.time() - start_time
        stages['vigenere'] = stage1

        # Stage 2: Substitution (preserves spaces)
        start_time = time.time()
        stage2 = self.substitution_encrypt(stage1, key)
        timing_data['substitution_encrypt'] = time.time() - start_time
        stages['substitution'] = stage2

        # Stage 3: Columnar Transposition (handles spaces with '_')
        start_time = time.time()
        final = self.columnar_transposition(stage2, key)
        timing_data['transposition_encrypt'] = time.time() - start_time
        stages['transposition'] = final
        stages['final'] = final

        # Total encryption time
        timing_data['total_encrypt'] = sum(timing_data.values())

        return final, stages, timing_data

    def decrypt(self, ciphertext, key, verbose=False):
        """Full decryption in reverse order"""
        stages = {}
        timing_data = {}

        # Reverse Stage 3: Columnar Transposition (restores spaces)
        start_time = time.time()
        stage1 = self.columnar_transposition_decrypt(ciphertext, key)
        timing_data['transposition_decrypt'] = time.time() - start_time
        stages['reverse_transposition'] = stage1

        # Reverse Stage 2: Substitution
        start_time = time.time()
        stage2 = self.substitution_decrypt(stage1, key)
        timing_data['substitution_decrypt'] = time.time() - start_time
        stages['reverse_substitution'] = stage2

        # Reverse Stage 1: Vigenère
        start_time = time.time()
        final = self.vigenere_decrypt(stage2, key)
        timing_data['vigenere_decrypt'] = time.time() - start_time
        stages['reverse_vigenere'] = final
        stages['final'] = final

        # Total decryption time
        timing_data['total_decrypt'] = sum(timing_data.values())

        return final, stages, timing_data


class CipherAnalysis:
    def __init__(self):
        self.english_freq = {
            'E': 12.70, 'T': 9.06, 'A': 8.17, 'O': 7.51, 'I': 6.97,
            'N': 6.75, 'S': 6.33, 'H': 6.09, 'R': 5.99, 'D': 4.25,
            'L': 4.03, 'C': 2.78, 'U': 2.76, 'M': 2.41, 'W': 2.36,
            'F': 2.23, 'G': 2.02, 'Y': 1.97, 'P': 1.93, 'B': 1.29,
            'V': 0.98, 'K': 0.77, 'J': 0.15, 'X': 0.15, 'Q': 0.10, 'Z': 0.07
        }

    def frequency_analysis_attack(self, ciphertext):
        """Perform frequency analysis to break the cipher"""
        start_time = time.time()

        # Clean ciphertext (ignore underscores used for spaces)
        clean_ct = ''.join([c for c in ciphertext.upper() if c in string.ascii_uppercase])

        if not clean_ct:
            return None, None, None, 0

        # Calculate frequencies
        freq = Counter(clean_ct)
        total = len(clean_ct)
        ct_freq = {char: (count/total)*100 for char, count in freq.items()}

        # Calculate frequency difference for different shifts
        best_shift = 0
        best_diff = float('inf')

        for shift in range(26):
            diff = 0
            for ct_char, ct_freq_val in ct_freq.items():
                plain_idx = (string.ascii_uppercase.index(ct_char) - shift) % 26
                plain_char = string.ascii_uppercase[plain_idx]
                english_freq_val = self.english_freq.get(plain_char, 0)
                diff += abs(ct_freq_val - english_freq_val)

            if diff < best_diff:
                best_diff = diff
                best_shift = shift

        # Try decryption with best shift
        attempted_decrypt = ''
        for char in ciphertext.upper():
            if char in string.ascii_uppercase:
                attempted_decrypt += string.ascii_uppercase[(string.ascii_uppercase.index(char) - best_shift) % 26]
            else:
                attempted_decrypt += char

        attack_time = time.time() - start_time
        return attempted_decrypt, best_shift, ct_freq, attack_time

    def brute_force_attack(self, ciphertext, max_key_length=5):
        """Brute force attack with timing measurements"""
        start_time = time.time()

        cipher = CustomCipher()
        results = []
        tested_keys = 0

        # Test common keys and short keys
        common_keys = ["SECRET", "KEY", "PASSWORD", "ENCRYPT", "CIPHER"]

        for key in common_keys:
            if len(key) >= 10:
                tested_keys += 1
                try:
                    decrypted, _, _ = cipher.decrypt(ciphertext, key)
                    results.append((key, decrypted))
                except:
                    pass

        # Test short keys (even though they're invalid for full decryption)
        for key_length in range(1, max_key_length + 1):
            for key_tuple in itertools.product(string.ascii_uppercase, repeat=key_length):
                if tested_keys >= 1000:  # Limit for performance
                    break
                key = ''.join(key_tuple)
                tested_keys += 1
                try:
                    decrypted, _, _ = cipher.decrypt(ciphertext, key + "X" * (10 - key_length))
                    results.append((key, decrypted))
                except:
                    pass

        attack_time = time.time() - start_time
        return results, tested_keys, attack_time

    def known_plaintext_attack(self, ciphertext, known_plaintext):
        """Known plaintext attack with timing"""
        start_time = time.time()

        cipher = CustomCipher()
        possible_keys = []

        # Try to find keys that produce the known plaintext from ciphertext
        for test_key in ["TESTKEY12345", "KNOWNKEY1234", "ATTACKKEY123"]:
            try:
                decrypted, _, _ = cipher.decrypt(ciphertext, test_key)
                if known_plaintext.upper() in decrypted.upper():
                    possible_keys.append(test_key)
            except:
                pass

        attack_time = time.time() - start_time
        return possible_keys, attack_time

    def create_frequency_plot(self, ct_freq):
        """Create a frequency comparison plot"""
        if not ct_freq:
            return None

        # Prepare data for plotting
        chars = list(self.english_freq.keys())
        english_values = [self.english_freq[char] for char in chars]
        cipher_values = [ct_freq.get(char, 0) for char in chars]

        fig = go.Figure()
        fig.add_trace(go.Bar(x=chars, y=english_values, name='English Frequency', marker_color='blue'))
        fig.add_trace(go.Bar(x=chars, y=cipher_values, name='Ciphertext Frequency', marker_color='red'))

        fig.update_layout(
            title='Frequency Analysis: English vs Ciphertext',
            xaxis_title='Letters',
            yaxis_title='Frequency (%)',
            barmode='group',
            height=400
        )

        return fig


class PerformanceComparator:
    def __init__(self):
        self.cipher = CustomCipher()

    def shift_cipher_encrypt(self, text, shift=3):
        """Simple shift cipher for comparison"""
        result = []
        for char in text.upper():
            if char in string.ascii_uppercase:
                result.append(string.ascii_uppercase[(string.ascii_uppercase.index(char) + shift) % 26])
            else:
                result.append(char)
        return ''.join(result)

    def shift_cipher_decrypt(self, text, shift=3):
        """Simple shift cipher decryption"""
        result = []
        for char in text.upper():
            if char in string.ascii_uppercase:
                result.append(string.ascii_uppercase[(string.ascii_uppercase.index(char) - shift) % 26])
            else:
                result.append(char)
        return ''.join(result)

    def compare_performance(self, text, key):
        """Compare performance between custom cipher and shift cipher"""
        if not text:
            return None, None, None

        timing_details = {}

        # Custom Cipher
        start_time = time.time()
        encrypted, stages, enc_timing = self.cipher.encrypt(text, key, verbose=False)
        custom_enc_time = time.time() - start_time
        timing_details['custom_encryption'] = enc_timing

        start_time = time.time()
        decrypted, stages, dec_timing = self.cipher.decrypt(encrypted, key, verbose=False)
        custom_dec_time = time.time() - start_time
        timing_details['custom_decryption'] = dec_timing

        # Shift Cipher
        start_time = time.time()
        shift_encrypted = self.shift_cipher_encrypt(text)
        shift_enc_time = time.time() - start_time

        start_time = time.time()
        shift_decrypted = self.shift_cipher_decrypt(shift_encrypted)
        shift_dec_time = time.time() - start_time

        # Attack timing analysis
        analyzer = CipherAnalysis()

        # Frequency analysis attack timing
        _, _, _, freq_attack_time = analyzer.frequency_analysis_attack(encrypted)
        timing_details['frequency_attack'] = freq_attack_time

        # Brute force attack timing (limited)
        _, tested_keys, brute_attack_time = analyzer.brute_force_attack(encrypted, max_key_length=3)
        timing_details['brute_force_attack'] = brute_attack_time
        timing_details['keys_tested'] = tested_keys

        # Known plaintext attack timing
        known_keys, known_attack_time = analyzer.known_plaintext_attack(encrypted, text[:5] if len(text) >= 5 else text)
        timing_details['known_plaintext_attack'] = known_attack_time
        timing_details['found_keys'] = len(known_keys)

        # Create performance comparison data
        performance_data = {
            'Cipher Type': ['Custom Cipher', 'Shift Cipher'],
            'Encryption Time (s)': [custom_enc_time, shift_enc_time],
            'Decryption Time (s)': [custom_dec_time, shift_dec_time]
        }

        # Create visualization
        fig = go.Figure()
        fig.add_trace(go.Bar(name='Encryption Time', x=performance_data['Cipher Type'], y=performance_data['Encryption Time (s)']))
        fig.add_trace(go.Bar(name='Decryption Time', x=performance_data['Cipher Type'], y=performance_data['Decryption Time (s)']))

        fig.update_layout(
            title='Performance Comparison: Custom Cipher vs Shift Cipher',
            yaxis_title='Time (seconds)',
            barmode='group',
            height=400
        )

        slowdown = custom_enc_time / shift_enc_time if shift_enc_time > 0 else 0

        return performance_data, fig, slowdown, timing_details


# Gradio Interface Functions
def encrypt_text(plaintext, key):
    """Encrypt the given text with detailed steps"""
    if not plaintext or not key:
        return "Please provide both plaintext and key", "", "", "", "", None, ""

    if len(key) < 10:
        return "Error: Key must be at least 10 characters long", "", "", "", "", None, ""

    cipher = CustomCipher()

    try:
        encrypted, stages, timing_data = cipher.encrypt(plaintext, key)

        # Create timing analysis HTML
        timing_html = f"""
        <div style='font-family: Arial, sans-serif; line-height: 1.6; background: #f0f8ff; padding: 15px; border-radius: 8px; margin: 10px 0;'>
            <h4 style='color: #2E86AB;'>⏱️ Encryption Timing Analysis</h4>
            <table style='width: 100%; border-collapse: collapse;'>
                <tr style='background: #e9ecef;'>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Stage</th>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Time (seconds)</th>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Vigenère Encryption</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_data['vigenere_encrypt']:.6f}</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Substitution Encryption</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_data['substitution_encrypt']:.6f}</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Transposition Encryption</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_data['transposition_encrypt']:.6f}</td>
                </tr>
                <tr style='background: #d4edda; font-weight: bold;'>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Total Encryption Time</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_data['total_encrypt']:.6f}</td>
                </tr>
            </table>
        </div>
        """

        # Create detailed steps output
        steps_html = f"""
        <div style='font-family: Arial, sans-serif; line-height: 1.6;'>
            <h3 style='color: #2E86AB;'>🔒 Encryption Process - Step by Step</h3>

            <div style='background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0;'>
                <h4 style='color: #A23B72;'>📝 Original Text:</h4>
                <p style='font-family: monospace; background: white; padding: 10px; border-radius: 4px;'>{plaintext}</p>
            </div>

            <div style='background: #e8f4f8; padding: 15px; border-radius: 8px; margin: 10px 0;'>
                <h4 style='color: #A23B72;'>🌀 Stage 1 - Vigenère Cipher:</h4>
                <p style='font-family: monospace; background: white; padding: 10px; border-radius: 4px;'>{stages['vigenere']}</p>
            </div>

            <div style='background: #e8f4f8; padding: 15px; border-radius: 8px; margin: 10px 0;'>
                <h4 style='color: #A23B72;'>🔄 Stage 2 - Substitution Cipher:</h4>
                <p style='font-family: monospace; background: white; padding: 10px; border-radius: 4px;'>{stages['substitution']}</p>
            </div>

            <div style='background: #e8f4f8; padding: 15px; border-radius: 8px; margin: 10px 0;'>
                <h4 style='color: #A23B72;'>📊 Stage 3 - Columnar Transposition:</h4>
                <p style='font-family: monospace; background: white; padding: 10px; border-radius: 4px;'>{stages['transposition']}</p>
            </div>

            <div style='background: #d4edda; padding: 15px; border-radius: 8px; margin: 10px 0;'>
                <h4 style='color: #155724;'>✅ Final Encrypted Text:</h4>
                <p style='font-family: monospace; background: white; padding: 10px; border-radius: 4px; font-weight: bold;'>{encrypted}</p>
            </div>
        </div>
        """

        return encrypted, stages['vigenere'], stages['substitution'], stages['transposition'], steps_html, encrypted, timing_html

    except Exception as e:
        return f"Error during encryption: {str(e)}", "", "", "", "", None, ""


def decrypt_text(ciphertext, key):
    """Decrypt the given text with detailed steps"""
    if not ciphertext or not key:
        return "Please provide both ciphertext and key", "", "", "", "", None, ""

    if len(key) < 10:
        return "Error: Key must be at least 10 characters long", "", "", "", "", None, ""

    cipher = CustomCipher()

    try:
        decrypted, stages, timing_data = cipher.decrypt(ciphertext, key)

        # Create timing analysis HTML
        timing_html = f"""
        <div style='font-family: Arial, sans-serif; line-height: 1.6; background: #f0f8ff; padding: 15px; border-radius: 8px; margin: 10px 0;'>
            <h4 style='color: #2E86AB;'>⏱️ Decryption Timing Analysis</h4>
            <table style='width: 100%; border-collapse: collapse;'>
                <tr style='background: #e9ecef;'>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Stage</th>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Time (seconds)</th>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Transposition Decryption</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_data['transposition_decrypt']:.6f}</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Substitution Decryption</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_data['substitution_decrypt']:.6f}</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Vigenère Decryption</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_data['vigenere_decrypt']:.6f}</td>
                </tr>
                <tr style='background: #d4edda; font-weight: bold;'>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Total Decryption Time</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_data['total_decrypt']:.6f}</td>
                </tr>
            </table>
        </div>
        """

        # Create detailed steps output
        steps_html = f"""
        <div style='font-family: Arial, sans-serif; line-height: 1.6;'>
            <h3 style='color: #2E86AB;'>🔓 Decryption Process - Step by Step</h3>

            <div style='background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0;'>
                <h4 style='color: #A23B72;'>📝 Encrypted Text:</h4>
                <p style='font-family: monospace; background: white; padding: 10px; border-radius: 4px;'>{ciphertext}</p>
            </div>

            <div style='background: #e8f4f8; padding: 15px; border-radius: 8px; margin: 10px 0;'>
                <h4 style='color: #A23B72;'>🌀 Reverse Stage 1 - Columnar Transposition:</h4>
                <p style='font-family: monospace; background: white; padding: 10px; border-radius: 4px;'>{stages['reverse_transposition']}</p>
            </div>

            <div style='background: #e8f4f8; padding: 15px; border-radius: 8px; margin: 10px 0;'>
                <h4 style='color: #A23B72;'>🔄 Reverse Stage 2 - Substitution Cipher:</h4>
                <p style='font-family: monospace; background: white; padding: 10px; border-radius: 4px;'>{stages['reverse_substitution']}</p>
            </div>

            <div style='background: #e8f4f8; padding: 15px; border-radius: 8px; margin: 10px 0;'>
                <h4 style='color: #A23B72;'>📊 Reverse Stage 3 - Vigenère Cipher:</h4>
                <p style='font-family: monospace; background: white; padding: 10px; border-radius: 4px;'>{stages['reverse_vigenere']}</p>
            </div>

            <div style='background: #d4edda; padding: 15px; border-radius: 8px; margin: 10px 0;'>
                <h4 style='color: #155724;'>✅ Final Decrypted Text:</h4>
                <p style='font-family: monospace; background: white; padding: 10px; border-radius: 4px; font-weight: bold;'>{decrypted}</p>
            </div>
        </div>
        """

        return decrypted, stages['reverse_transposition'], stages['reverse_substitution'], stages['reverse_vigenere'], steps_html, decrypted, timing_html

    except Exception as e:
        return f"Error during decryption: {str(e)}", "", "", "", "", None, ""


def analyze_ciphertext(ciphertext):
    """Perform frequency analysis on ciphertext"""
    if not ciphertext:
        return "Please provide ciphertext to analyze", None, None, None, ""

    analyzer = CipherAnalysis()
    attempted_decrypt, best_shift, ct_freq, attack_time = analyzer.frequency_analysis_attack(ciphertext)

    if attempted_decrypt is None:
        return "No alphabetic characters found for analysis", None, None, None, ""

    # Create frequency plot
    freq_plot = analyzer.create_frequency_plot(ct_freq)

    # Perform other attacks for comparison
    brute_results, tested_keys, brute_time = analyzer.brute_force_attack(ciphertext, max_key_length=3)
    known_keys, known_time = analyzer.known_plaintext_attack(ciphertext, "THE" if len(ciphertext) > 3 else ciphertext)

    timing_html = f"""
    <div style='font-family: Arial, sans-serif; line-height: 1.6; background: #fff3cd; padding: 15px; border-radius: 8px; margin: 10px 0;'>
        <h4 style='color: #856404;'>⏱️ Attack Timing Analysis</h4>
        <table style='width: 100%; border-collapse: collapse;'>
            <tr style='background: #e9ecef;'>
                <th style='padding: 8px; border: 1px solid #ddd;'>Attack Type</th>
                <th style='padding: 8px; border: 1px solid #ddd;'>Time (seconds)</th>
                <th style='padding: 8px; border: 1px solid #ddd;'>Additional Info</th>
            </tr>
            <tr>
                <td style='padding: 8px; border: 1px solid #ddd;'>Frequency Analysis</td>
                <td style='padding: 8px; border: 1px solid #ddd;'>{attack_time:.6f}</td>
                <td style='padding: 8px; border: 1px solid #ddd;'>Best shift: {best_shift}</td>
            </tr>
            <tr>
                <td style='padding: 8px; border: 1px solid #ddd;'>Brute Force (Limited)</td>
                <td style='padding: 8px; border: 1px solid #ddd;'>{brute_time:.6f}</td>
                <td style='padding: 8px; border: 1px solid #ddd;'>{tested_keys} keys tested</td>
            </tr>
            <tr>
                <td style='padding: 8px; border: 1px solid #ddd;'>Known Plaintext</td>
                <td style='padding: 8px; border: 1px solid #ddd;'>{known_time:.6f}</td>
                <td style='padding: 8px; border: 1px solid #ddd;'>{len(known_keys)} possible keys found</td>
            </tr>
        </table>
        <p style='margin-top: 10px; font-size: 0.9em; color: #856404;'>
            Note: Brute force attack is limited to short keys for performance reasons. Full brute force would take exponentially longer.
        </p>
    </div>
    """

    analysis_html = f"""
    <div style='font-family: Arial, sans-serif; line-height: 1.6;'>
        <h3 style='color: #2E86AB;'>🔍 Frequency Analysis Results</h3>

        <div style='background: #fff3cd; padding: 15px; border-radius: 8px; margin: 10px 0;'>
            <h4 style='color: #856404;'>🎯 Best Shift Found: {best_shift}</h4>
            <p>This shift minimizes the difference between ciphertext frequencies and English language frequencies.</p>
        </div>

        <div style='background: #e8f4f8; padding: 15px; border-radius: 8px; margin: 10px 0;'>
            <h4 style='color: #A23B72;'>🔓 Attempted Decryption with Shift {best_shift}:</h4>
            <p style='font-family: monospace; background: white; padding: 10px; border-radius: 4px;'>{attempted_decrypt}</p>
        </div>

        <div style='background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0;'>
            <h4 style='color: #A23B72;'>📊 Analysis Note:</h4>
            <p>This attack works best on simple substitution ciphers. The multi-stage encryption used here provides stronger protection against frequency analysis.</p>
        </div>
    </div>
    """

    return analysis_html, freq_plot, attempted_decrypt, best_shift, timing_html


def compare_performance_interface(text, key):
    """Compare performance between ciphers"""
    if not text or not key:
        return "Please provide both text and key for performance comparison", None, None, ""

    comparator = PerformanceComparator()
    performance_data, performance_plot, slowdown, timing_details = comparator.compare_performance(text, key)

    if performance_data is None:
        return "Error in performance comparison", None, None, ""

    # Create detailed timing analysis
    timing_html = f"""
    <div style='font-family: Arial, sans-serif; line-height: 1.6;'>
        <h3 style='color: #2E86AB;'>⏱️ Detailed Timing Analysis</h3>

        <div style='background: #e8f4f8; padding: 15px; border-radius: 8px; margin: 10px 0;'>
            <h4 style='color: #A23B72;'>🔒 Custom Cipher Timing Breakdown:</h4>
            <table style='width: 100%; border-collapse: collapse;'>
                <tr style='background: #e9ecef;'>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Encryption Stage</th>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Time (seconds)</th>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Vigenère</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_details['custom_encryption']['vigenere_encrypt']:.6f}</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Substitution</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_details['custom_encryption']['substitution_encrypt']:.6f}</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Transposition</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_details['custom_encryption']['transposition_encrypt']:.6f}</td>
                </tr>
                <tr style='background: #d4edda; font-weight: bold;'>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Total Encryption</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_details['custom_encryption']['total_encrypt']:.6f}</td>
                </tr>
            </table>
        </div>

        <div style='background: #fff3cd; padding: 15px; border-radius: 8px; margin: 10px 0;'>
            <h4 style='color: #856404;'>🛡️ Attack Timing Analysis:</h4>
            <table style='width: 100%; border-collapse: collapse;'>
                <tr style='background: #e9ecef;'>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Attack Type</th>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Time (seconds)</th>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Notes</th>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Frequency Analysis</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_details['frequency_attack']:.6f}</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Fast but limited effectiveness</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Brute Force (Limited)</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_details['brute_force_attack']:.6f}</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_details['keys_tested']} keys tested</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Known Plaintext</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_details['known_plaintext_attack']:.6f}</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{timing_details['found_keys']} keys found</td>
                </tr>
            </table>
        </div>
    </div>
    """

    performance_html = f"""
    <div style='font-family: Arial, sans-serif; line-height: 1.6;'>
        <h3 style='color: #2E86AB;'>⚡ Performance Comparison Results</h3>

        <div style='background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0;'>
            <h4 style='color: #A23B72;'>📈 Performance Metrics:</h4>
            <table style='width: 100%; border-collapse: collapse;'>
                <tr style='background: #e9ecef;'>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Cipher Type</th>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Encryption Time</th>
                    <th style='padding: 8px; border: 1px solid #ddd;'>Decryption Time</th>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Custom Cipher</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{performance_data['Encryption Time (s)'][0]:.6f}s</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{performance_data['Decryption Time (s)'][0]:.6f}s</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Shift Cipher</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{performance_data['Encryption Time (s)'][1]:.6f}s</td>
                    <td style='padding: 8px; border: 1px solid #ddd;'>{performance_data['Decryption Time (s)'][1]:.6f}s</td>
                </tr>
            </table>
        </div>

        <div style='background: #fff3cd; padding: 15px; border-radius: 8px; margin: 10px 0;'>
            <h4 style='color: #856404;'>📊 Performance Overhead:</h4>
            <p>The custom multi-stage cipher is <strong>{slowdown:.2f}x slower</strong> than the simple shift cipher.</p>
            <p>This overhead is the trade-off for enhanced security through multiple encryption layers.</p>
        </div>
    </div>
    """

    return performance_html, performance_plot, slowdown, timing_html


# Create Gradio Interface
def create_interface():
    """Create the main Gradio interface"""

    with gr.Blocks(theme=gr.themes.Soft(), title="Advanced Multi-Stage Cipher System") as demo:
        gr.Markdown(
            """
            # 🔐 Advanced Multi-Stage Cipher System
            ### *Vigenère + Substitution + Columnar Transposition*

            This system combines three classical encryption techniques for enhanced security:
            - **Stage 1**: Vigenère Cipher (Polyalphabetic substitution)
            - **Stage 2**: Substitution Cipher (Monoalphabetic substitution)
            - **Stage 3**: Columnar Transposition (Position-based permutation)

            **🔑 Key Requirement**: Minimum 10 characters
            """
        )

        with gr.Tabs():
            # Tab 1: Encryption
            with gr.TabItem("🔒 Encryption"):
                with gr.Row():
                    with gr.Column():
                        encrypt_input = gr.Textbox(
                            label="Plaintext to Encrypt",
                            placeholder="Enter your secret message here...",
                            lines=3
                        )
                        encrypt_key = gr.Textbox(
                            label="Encryption Key",
                            placeholder="Enter a key (min 10 characters)...",
                            type="password"
                        )
                        encrypt_btn = gr.Button("🚀 Encrypt Text", variant="primary")

                    with gr.Column():
                        encrypted_output = gr.Textbox(
                            label="Final Encrypted Text",
                            interactive=False,
                            lines=3
                        )
                        encrypt_steps = gr.HTML(label="Encryption Steps")

                with gr.Row():
                    with gr.Column():
                        gr.Markdown("### 🔄 Intermediate Steps")
                        vigenere_step = gr.Textbox(label="After Vigenère Cipher", interactive=False)
                        substitution_step = gr.Textbox(label="After Substitution Cipher", interactive=False)
                        transposition_step = gr.Textbox(label="After Columnar Transposition", interactive=False)

                with gr.Row():
                    encrypt_timing = gr.HTML(label="⏱️ Encryption Timing Analysis")

            # Tab 2: Decryption
            with gr.TabItem("🔓 Decryption"):
                with gr.Row():
                    with gr.Column():
                        decrypt_input = gr.Textbox(
                            label="Ciphertext to Decrypt",
                            placeholder="Enter encrypted text here...",
                            lines=3
                        )
                        decrypt_key = gr.Textbox(
                            label="Decryption Key",
                            placeholder="Enter the same key used for encryption...",
                            type="password"
                        )
                        decrypt_btn = gr.Button("🔍 Decrypt Text", variant="primary")

                    with gr.Column():
                        decrypted_output = gr.Textbox(
                            label="Final Decrypted Text",
                            interactive=False,
                            lines=3
                        )
                        decrypt_steps = gr.HTML(label="Decryption Steps")

                with gr.Row():
                    with gr.Column():
                        gr.Markdown("### 🔄 Reverse Steps")
                        reverse_transposition = gr.Textbox(label="After Reverse Transposition", interactive=False)
                        reverse_substitution = gr.Textbox(label="After Reverse Substitution", interactive=False)
                        reverse_vigenere = gr.Textbox(label="After Reverse Vigenère", interactive=False)

                with gr.Row():
                    decrypt_timing = gr.HTML(label="⏱️ Decryption Timing Analysis")

            # Tab 3: Security Analysis
            with gr.TabItem("🔍 Security Analysis"):
                with gr.Row():
                    with gr.Column():
                        analysis_input = gr.Textbox(
                            label="Ciphertext to Analyze",
                            placeholder="Enter ciphertext for frequency analysis...",
                            lines=3
                        )
                        analyze_btn = gr.Button("📊 Analyze Ciphertext", variant="primary")

                    with gr.Column():
                        analysis_results = gr.HTML(label="Analysis Results")
                        freq_plot = gr.Plot(label="Frequency Comparison")

                with gr.Row():
                    analysis_attempt = gr.Textbox(
                        label="Attempted Decryption (Frequency Analysis)",
                        interactive=False,
                        lines=2
                    )
                    best_shift = gr.Number(
                        label="Best Shift Found",
                        interactive=False
                    )

                with gr.Row():
                    attack_timing = gr.HTML(label="⏱️ Attack Timing Analysis")

            # Tab 4: Performance Comparison
            with gr.TabItem("⚡ Performance"):
                with gr.Row():
                    with gr.Column():
                        perf_text = gr.Textbox(
                            label="Text for Performance Test",
                            placeholder="Enter text to compare performance...",
                            lines=2
                        )
                        perf_key = gr.Textbox(
                            label="Key for Performance Test",
                            placeholder="Enter key for performance test...",
                            type="password"
                        )
                        perf_btn = gr.Button("⏱️ Compare Performance", variant="primary")

                    with gr.Column():
                        perf_results = gr.HTML(label="Performance Results")
                        perf_plot = gr.Plot(label="Performance Comparison Chart")

                with gr.Row():
                    slowdown_factor = gr.Number(
                        label="Slowdown Factor (Custom vs Shift Cipher)",
                        interactive=False
                    )

                with gr.Row():
                    detailed_timing = gr.HTML(label="⏱️ Detailed Timing Analysis")

        # Event handlers
        encrypt_btn.click(
            encrypt_text,
            inputs=[encrypt_input, encrypt_key],
            outputs=[encrypted_output, vigenere_step, substitution_step, transposition_step, encrypt_steps, analysis_input, encrypt_timing]
        )

        decrypt_btn.click(
            decrypt_text,
            inputs=[decrypt_input, decrypt_key],
            outputs=[decrypted_output, reverse_transposition, reverse_substitution, reverse_vigenere, decrypt_steps, analysis_input, decrypt_timing]
        )

        analyze_btn.click(
            analyze_ciphertext,
            inputs=[analysis_input],
            outputs=[analysis_results, freq_plot, analysis_attempt, best_shift, attack_timing]
        )

        perf_btn.click(
            compare_performance_interface,
            inputs=[perf_text, perf_key],
            outputs=[perf_results, perf_plot, slowdown_factor, detailed_timing]
        )

        # Examples and instructions
        gr.Markdown(
            """
            ## 💡 Usage Examples

            **Example 1: Basic Encryption**
            - Plaintext: `HELLO WORLD`
            - Key: `SECRETKEY123456789`

            **Example 2: Longer Message**
            - Plaintext: `ATTACK AT DAWN`
            - Key: `MYENCRYPTIONKEY123`

            ## 🛡️ Security Features

            - **Multi-layer encryption** making cryptanalysis difficult
            - **Key-dependent substitution** patterns
            - **Transposition** breaking frequency patterns
            - **Minimum 10-character key** requirement

            ## 📊 Technical Details

            The system implements:
            - **Vigenère Cipher**: Polyalphabetic substitution using key
            - **Substitution Cipher**: Key-based alphabet permutation
            - **Columnar Transposition**: Position-based character shuffling
            - **Combined Security**: O(n) time complexity with enhanced protection
            """
        )

    return demo

# Launch the interface
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://89cf2840fc009852b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
